In [1]:
import json
from data_adaptor import DataAdaptor

/opt/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
/opt/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max

## 2WikiMultihopQA
Authors only use question-answer information. The context is not provided.
> "we use only the question-answer pairs from these datasets, not any passages of relevant text that they contain. These datasets both contain 2-hop compositional questions sourced from facts that appear in Wikipedia articles." - Press, et al.

Authors do not use this dataset to measure compositionality gap which requires known sub-questions and answers to measure.
> "Note that the rest of this section shows that elicitive prompts improve performance but does not show that they narrow the compositionality gap since we lack sub-questions for datasets other than CC." - Press, et al.

In [2]:
from data_loaders import load_2WikiMultihopQA

In [3]:
# Load the training data
wiki_sample = load_2WikiMultihopQA(n_examples=5, split='train')

In [4]:
type(wiki_sample)

list

In [17]:
# Print the first example in pretty format
print(json.dumps(wiki_sample[0], indent=4))

{
    "_id": "8813f87c0bdd11eba7f7acde48001122",
    "type": "compositional",
    "question": "Who is the mother of the director of film Polish-Russian War (Film)?",
    "context": [
        [
            "Maheen Khan",
            [
                "Maheen Khan is a Pakistani fashion and costume designer, also an award winner fashion designer for fashion labels like\" The Embroidery HouseMaheen\" and\" Gulabo\".",
                "She has done many national and international fashion events and shows.",
                "She undertook embroidery for the film Snow White and the Huntsman and television series",
                "The Jewel in the Crown."
            ]
        ],
        [
            "Viktor Yeliseyev",
            [
                "Viktor Petrovich Yeliseyev( born June 9, 1950) is a Russian general, orchestra conductor and music teacher.",
                "He is the director of the Ministry of the Interior Ensemble, one of the two Russian Red Army Choirs."
            ]
 

Let's look at the evidence required to answer the question. It's possible we create prompt examples using these evidences to insert into the fine-tuning set. For example,

**Examplar:**
```
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg.
Follow up: Who is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no
```

In [4]:
wiki_sample[0]['supporting_facts']

[['Move (1970 film)', 0],
 ['Méditerranée (1963 film)', 0],
 ['Stuart Rosenberg', 0],
 ['Jean-Daniel Pollet', 0]]

### Adapting to Self-Ask Examplar

In [4]:
wiki_adaptor = DataAdaptor(dataset="2WikiMultihopQA")

In [5]:
wiki_examplars = wiki_adaptor.generate_examplars(wiki_sample, strategy="self-ask")
for examplar in wiki_examplars:
    print(examplar)

2023-06-29 13:12:36.475 | DEBUG    | data_adaptor:_compose_2WikiMultihopQA_subquestions:348 - Could not find entity type for Rune Gerhardsen.


Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
Intermediate answer: Croatian
Follow

### Adapting to Self-Ask Training Example
We can augment the target texts in the dataset with the self-ask rationale to fine-tune a language model to generate text with the self-ask rationale.

In [7]:
wiki_training_examples = wiki_adaptor.generate_training_examples(wiki_sample, strategy="self-ask")
for training_example in wiki_training_examples:
    print(json.dumps(training_example, indent=4))

2023-06-29 12:44:10.745 | DEBUG    | data_adaptor:_compose_2WikiMultihopQA_subquestions:300 - Could not find entity type for Rune Gerhardsen.


{
    "prompt": "Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Genevi\u00e8ve Wa\u00efte, and directed by Stuart Rosenberg.\nFact #1: M\u00e9diterran\u00e9e is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schl\u00f6ndorff.\nFact #2: Stuart Rosenberg (August 11, 1927 \u2013 March 15, 2007) was an American film and television director whose motion pictures include \"Cool Hand Luke\" (1967), \"Voyage of the Damned\" (1976), \"The Amityville Horror\" (1979), and \"The Pope of Greenwich Village\" (1984).\nFact #3: Jean-Daniel Pollet (1936\u20132004) was a French film director and screenwriter who was most active in the 1960s and 1970s.\n\nQuestion: Are director of film Move (1970 Film) and director of film M\u00e9diterran\u00e9e (1963 Film) from the same country?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: Who is the director of Move (1970 film)?\nIntermediate answer: Stuart

In [8]:
print(wiki_training_examples[0]["prompt"])
print(wiki_training_examples[0]["target"])

Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg.
Fact #1: Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.
Fact #2: Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).
Fact #3: Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s.

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here:

Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jea

### Direct Prompting Training Examples
Simply provide the facts and ask the question. No thought variable or rationale involved.

In [6]:
direct_training_examples = wiki_adaptor.generate_training_examples(
    wiki_sample[0],
    strategy="direct"
)

In [7]:
print("--------- Augmented Prompt ---------")
print(direct_training_examples[0]["prompt"])
print("--------- Target ---------")
print(direct_training_examples[0]["target"])

--------- Augmented Prompt ---------
Fact #0: Move is a 1970 American comedy film starring Elliott Gould, Paula Prentiss and Geneviève Waïte, and directed by Stuart Rosenberg.
Fact #1: Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.
Fact #2: Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).
Fact #3: Jean-Daniel Pollet (1936–2004) was a French film director and screenwriter who was most active in the 1960s and 1970s.

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Answer:
--------- Target ---------
no


### Augment with In-Context Examplars and Self-Ask Rationale Targets
We can combine the two above to create an augmented fine-tuning dataset:
1. Prompt text has in-context examplars
2. Target text has the self-ask rationale

In [7]:
training_examplars = wiki_examplars[:4]
augmented_example = wiki_adaptor.generate_training_examples(
    wiki_sample[0], 
    strategy="self-ask", 
    examplars=training_examplars
    )[0]
print("--------- Augmented Prompt ---------")
print(augmented_example["prompt"])
print("--------- Target ---------")
print(augmented_example["target"])

--------- Augmented Prompt ---------
Example Response
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Example Response
Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country o

In [8]:
training_examplars = wiki_examplars[:2]
augmented_examples = wiki_adaptor.generate_training_examples(
    wiki_sample, 
    strategy="self-ask",
    examplars=training_examplars
    )

# look at token counts in prompt (context size)
print("context size")
for example in augmented_examples:
    print(example["num_prompt_tokens"])

# look at token counts
print("total tokens")
for example in augmented_examples:
    print(example["num_tokens"])

2023-06-29 12:53:24.922 | DEBUG    | data_adaptor:_compose_2WikiMultihopQA_subquestions:304 - Could not find entity type for Rune Gerhardsen.


context size
517
532
461
374
433
total tokens
617
653
558
432
496


In [9]:
print(augmented_examples[1]["prompt"])

Example Response
Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of Move (1970 film)?
Intermediate answer: Stuart Rosenberg
Follow up: What is the director of Méditerranée (1963 film)?
Intermediate answer: Jean-Daniel Pollet
Follow up: What is the country of citizenship of Stuart Rosenberg?
Intermediate answer: American
Follow up: What is the country of citizenship of Jean-Daniel Pollet?
Intermediate answer: French
So the final answer is: no

Example Response
Question: Do both films The Falcon (Film) and Valentin The Good have the directors from the same country?
Are follow up questions needed here: Yes.
Follow up: Who is the director of The Falcon (film)?
Intermediate answer: Vatroslav Mimica
Follow up: Who is the director of Valentin the Good?
Intermediate answer: Martin Frič
Follow up: What is the country of citizenship of Vatroslav Mimica?
In

## Compositional Celebrities

In [1]:
from data_loaders import load_CompositionalCelebrities

In [3]:
cc_sample = load_CompositionalCelebrities(n_examples=5, split='train')

/Users/richardmathews/Downloads/W266 project/data_loaders.py:54: UserWarning: There is only one file. The data has not been splitted yet.
  warnings.warn("There is only one file. The data has not been splitted yet.")


Let's take a look at an example. We see the subquestions are clearly delineated, with answers, as well as the compositional question itself. The other metadata is not necessary for our research.

In [4]:
# Print the first example in pretty format
print(json.dumps(cc_sample[0], indent=4))

{
    "Q1": "What is the birthplace (country only) of Rumi?",
    "A1": [
        "Afghanistan"
    ],
    "Q2": "What is the capital of Afghanistan?",
    "A2": [
        "Kabul"
    ],
    "Question": "What is the capital of the birthplace of Rumi?",
    "Answer": [
        "Kabul"
    ],
    "birth_country": "Afghanistan",
    "capital": [
        "Kabul"
    ],
    "person": "Rumi",
    "category": "birthplace_capital",
    "person_id": 0
}


### Adapting to Self-Ask Examplar
It's quite simple to restructure this as a "self-ask" examplar for prompting/fine-tuning.

In [5]:
cc_adaptor = DataAdaptor(dataset="CompositionalCelebrities")

In [6]:
cc_examplars = cc_adaptor.generate_examplars(cc_sample, strategy="self-ask")
for examplar in cc_examplars:
    print(examplar)

Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah Massoud?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Ahmad Shah Massoud?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Mahmud of Ghazni?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Mahmud of Ghazni?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah D

### Adapting to Self-Ask Training Example
We can augment the target texts in the dataset with the self-ask rationale to fine-tune a language model to generate text with the self-ask rationale.

In [7]:
cc_training_examples = cc_adaptor.generate_training_examples(cc_sample, strategy="self-ask")
for training_example in cc_training_examples:
    print(json.dumps(training_example, indent=4))

{
    "prompt": "Question: What is the capital of the birthplace of Rumi?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Rumi?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Afghanistan?\nIntermediate answer: Kabul\nSo the final answer is: Kabul\n"
}
{
    "prompt": "Question: What is the capital of the birthplace of Ahmad Shah Massoud?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Ahmad Shah Massoud?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Afghanistan?\nIntermediate answer: Kabul\nSo the final answer is: Kabul\n"
}
{
    "prompt": "Question: What is the capital of the birthplace of Mahmud of Ghazni?\nAre follow up questions needed here:\n",
    "target": "Yes.\nFollow up: What is the birthplace (country only) of Mahmud of Ghazni?\nIntermediate answer: Afghanistan\nFollow up: What is the capital of Af

In [8]:
print(cc_training_examples[0]["prompt"])
print(cc_training_examples[0]["target"])

Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here:

Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul



### Augment with In-Context Examplars and Self-Ask Rationale Targets
We can combine the two above to create an augmented fine-tuning dataset:
1. Prompt text has in-context examplars
2. Target text has the self-ask rationale

In [10]:
training_examplars = cc_examplars[:4]
augmented_example = {"prompt" : "\n".join(training_examplars) + "\n" + cc_training_examples[4]["prompt"], "target" : cc_training_examples[4]["target"]}
print("--------- Augmented Prompt ---------")
print(augmented_example["prompt"])
print("--------- Target ---------")
print(augmented_example["target"])

--------- Augmented Prompt ---------
Question: What is the capital of the birthplace of Rumi?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Rumi?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Ahmad Shah Massoud?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Ahmad Shah Massoud?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capital of the birthplace of Mahmud of Ghazni?
Are follow up questions needed here: Yes.
Follow up: What is the birthplace (country only) of Mahmud of Ghazni?
Intermediate answer: Afghanistan
Follow up: What is the capital of Afghanistan?
Intermediate answer: Kabul
So the final answer is: Kabul

Question: What is the capi